<a href="https://colab.research.google.com/github/RashiAGoyal/Best_Pick/blob/main/Best_picks_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195799 sha256=ff790d81da36764c12c38f3575381e15e8d0dfc1078cc20236b094a6ec6526ec
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from surprise import Dataset 
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy


In [ ]:
items=pd.read_csv('/content/drive/MyDrive/Trigital /prep_items_202303291533.csv')

In [ ]:
items.item_name =items.item_name.str.rstrip()

In [ ]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10142 entries, 0 to 10141
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10142 non-null  int64  
 1   item_id              0 non-null      float64
 2   tax_code             0 non-null      float64
 3   is_deleted           0 non-null      float64
 4   tenant_id            0 non-null      float64
 5   provider_id          0 non-null      float64
 6   catalog_id           0 non-null      float64
 7   is_availabe          0 non-null      float64
 8   dwh_updated_at       0 non-null      float64
 9   _id                  10142 non-null  object 
 10  description_summary  10142 non-null  object 
 11  provider             10142 non-null  object 
 12  discount             3597 non-null   float64
 13  discountpercent      4830 non-null   object 
 14  dealprice            8773 non-null   object 
 15  price                10139 non-null 

In [ ]:
items.item_name.value_counts()

Name                                              489
CAIRON WHITE SATIN SOLID CLUBWEAR SHIRT            50
CAIRON GREY SATIN SOLID CLUBWEAR SHIRT             48
CAIRON RED CHECK CASUAL SHIRT                      37
Bigen B103 Men's Beard Haircolor                   35
                                                 ... 
Lipton Green Tea Bags Honey Lemon, 10 N             1
Tetley Green Tea Bag Ginger, Mint Lemon, 25 N       1
Lipton Lemon Honey Green Tea Bags Sachet, 25 N      1
Lipton Green Tea 100 g                              1
Diamond mirshi 100g                                 1
Name: item_name, Length: 3796, dtype: int64

In [ ]:
order=pd.read_csv('/content/drive/MyDrive/Trigital /prep_orderitem_dummy_202303291450.csv')

In [ ]:
customer=pd.read_csv('/content/drive/MyDrive/Trigital /prep_customers_address_202304031410.csv')

In [ ]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11604 entries, 0 to 11603
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _id          11604 non-null  object 
 1   customerno   11604 non-null  object 
 2   tenant       11604 non-null  object 
 3   address_id   11604 non-null  object 
 4   isdefault    11604 non-null  bool   
 5   latitude     0 non-null      float64
 6   longitude    0 non-null      float64
 7   type         11604 non-null  object 
 8   city         11604 non-null  object 
 9   country      11604 non-null  object 
 10  housenumber  11602 non-null  object 
 11  state        11604 non-null  object 
 12  street1      11603 non-null  object 
 13  street2      11604 non-null  object 
 14  zip          11604 non-null  object 
 15  category     11604 non-null  object 
 16  fullname     11604 non-null  object 
 17  gender       11604 non-null  object 
 18  age          11604 non-null  int64  
dtypes: b

In [ ]:
customer.state.value_counts()

Telangana          6003
Andhra Pradesh     3114
Telangana           880
West Bengal         198
Andhra Pradesh      140
                   ... 
ANDHRA Pradesh        1
Igigitgi              1
ODISHA                1
500038                1
Nalgonda              1
Name: state, Length: 313, dtype: int64

In [ ]:
customer.state=customer.state.str.replace(' ', '')

In [ ]:
customer.state=customer.state.map({'Telangana':'Telangana','Telagna':'Telagna','WestBengal':'West Bengal',
                                       'Andhara Pradesh':'Andhara Pradesh','Gujarat':'Gujarat',
                                   'Andrapradesh':'Andhara Pradesh','AndhraPradesh':'Andhara Pradesh',
                                   'Avadi': 'Andhara Pradesh','Telanganana':'Telangana','Rajasthan':'Rajasthan'})

In [ ]:
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   item_id     10000 non-null  object
 1   customerid  9892 non-null   object
 2   serialno    10000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [ ]:
df_merged =items.merge(order,how='left',left_on='_id',right_on='item_id',suffixes=('', '_drop'))
df_merged.drop([col for col in df_merged.columns if 'drop' in col], axis=1, inplace=True)

In [ ]:
df_merged.market=df_merged.market.map({'Clothes & Fashion':'Clothes & Fashion','Grocery':'Grocery','Food & Restaurants':'Food & Restaurants',
                                       'Clothes  & Fashion':'Clothes & Fashion','Electronics':'Electronics',
                                   'Office & Stationary':'Office & Stationary','Office & Stationery':'Office & Stationary',
                                   'Home Improvement': 'Home Improvement','Virtual assets & Real Estate':'Virtual assets & Real Estate',
                                   'virtual assets & Real Estate':'Virtual assets & Real Estate','Fruits & Vegetables':'Fruits & Vegetables','Health Care':'Health Care'})

In [ ]:
df_merged.to_csv('df_merged.csv')

In [ ]:
df_merged.shape

(10142, 39)

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10142 entries, 0 to 10141
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10142 non-null  int64  
 1   item_id              0 non-null      float64
 2   tax_code             0 non-null      float64
 3   is_deleted           0 non-null      float64
 4   tenant_id            0 non-null      float64
 5   provider_id          0 non-null      float64
 6   catalog_id           0 non-null      float64
 7   is_availabe          0 non-null      float64
 8   dwh_updated_at       0 non-null      float64
 9   _id                  10142 non-null  object 
 10  description_summary  10142 non-null  object 
 11  provider             10142 non-null  object 
 12  discount             3597 non-null   float64
 13  discountpercent      4830 non-null   object 
 14  dealprice            8773 non-null   object 
 15  price                10139 non-null 

In [ ]:
data_merged =df_merged.merge(customer,how='left',left_on='customerid',right_on='_id',suffixes=('', '_drop'))
data_merged.drop([col for col in df_merged.columns if 'drop' in col], axis=1, inplace=True)

In [ ]:
data_merged.shape

(11895, 58)

In [ ]:
dict(data_merged.market.value_counts())

{'Clothes & Fashion': 7165,
 'Grocery': 2953,
 'Food & Restaurants': 1017,
 'Office & Stationary': 381,
 'Electronics': 336,
 'Home Improvement': 26,
 'Virtual assets & Real Estate': 15,
 'Fruits & Vegetables': 1,
 'Health Care': 1}

In [ ]:
final_df=data_merged[['customerid','id','gender','age','item_name','avgrating']]

In [ ]:
final_df.head()

,customerid,id,gender,age,item_name,avgrating
0,628cb023c3ccbfbb1d5f410e,1,Male,63.0,Chakki Fresh Atta,1
1,633ac3aff0f1d2a7c6a29f5a,7399,Male,63.0,Wings Biryani,1
2,630718b7cf57702ea522b0cb,754,Male,41.0,Jeera,4
3,63419766f0f1d24d99a34f54,7400,Male,63.0,Roti,4
4,62dbd5ce9670f2007f7cce70,809,Male,47.0,Nuts,3


In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11895 entries, 0 to 11894
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   customerid  11569 non-null  object 
 1   id          11895 non-null  int64  
 2   gender      8330 non-null   object 
 3   age         8330 non-null   float64
 4   item_name   11895 non-null  object 
 5   avgrating   11895 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 650.5+ KB


In [ ]:
final_df['gender'] = final_df['gender'].fillna(final_df['gender'].mode()[0])
final_df['age'] = final_df['age'].fillna(final_df['age'].mode()[0])

<ipython-input-28-de8ee009fd1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['gender'] = final_df['gender'].fillna(final_df['gender'].mode()[0])
<ipython-input-28-de8ee009fd1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age'] = final_df['age'].fillna(final_df['age'].mode()[0])


In [ ]:
final_df=final_df.dropna(subset=['customerid'])

In [ ]:
# label_encoder = preprocessing.LabelEncoder()
# final_df['customerid']= label_encoder.fit_transform(final_df['customerid'])

## Total number of customer, items and ratings

In [ ]:
print("Total no of ratings :",final_df.shape[0])
print("No. of unique customers:", final_df["customerid"].nunique())
print("No. of unique items:", final_df["id"].nunique())

Total no of ratings : 11569
No. of unique customers: 2388
No. of unique items: 9816


## Distribution of ratings

In [ ]:
final_df.avgrating.value_counts()

1    2372
2    2361
4    2337
3    2299
5    2200
Name: avgrating, dtype: int64

#  Ratings per user

In [ ]:
ratings_per_user = final_df.groupby(by='customerid')['avgrating'].count()
ratings_per_user.describe()

count    2388.000000
mean        4.844640
std        18.560868
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max       635.000000
Name: avgrating, dtype: float64

## Ratings per items

In [ ]:
final_df.customerid.value_counts()

62d7e395ec76434350c3b45c    635
633434892288e00638e17613    361
62b7faa283be59b149130ba3    238
62dbd8ab9670f2007f7d3eef    180
63353e88f0f1d27889a1f733    154
                           ... 
633320ca2288e0a136e1420d      1
62eb65a1a636f0a43d62194a      1
62dbd7149670f2007f7d018e      1
62dd470dec7643856cc3ca14      1
634d0bb9c4fba05c88231703      1
Name: customerid, Length: 2388, dtype: int64

In [ ]:
ratings_per_item = final_df.groupby(by='id')['avgrating'].count()
ratings_per_item.describe()

count    9816.000000
mean        1.178586
std         1.232930
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        72.000000
Name: avgrating, dtype: float64

# Loading as Surprise dataframe and train-test split

In [ ]:
reader = Reader()
ratings = Dataset.load_from_df(final_df[['customerid', 'id', 'avgrating']], reader)

In [ ]:
train_ratings, test_ratings = train_test_split(ratings, test_size=.20, random_state = 42)
print("Size of trainset: ", train_ratings.n_ratings)
print("Size of testset: ", len(test_ratings))

Size of trainset:  9255
Size of testset:  2314


In [ ]:
from surprise import BaselineOnly

In [ ]:
baseline_model = BaselineOnly(verbose = False)
baseline_model.fit(train_ratings)

In [ ]:
train_predictions = baseline_model.test(train_ratings.build_testset())
test_predictions = baseline_model.test(test_ratings)
print("RMSE on training data : ", accuracy.rmse(train_predictions,verbose = False))
print("RMSE on test data: ", accuracy.rmse(test_predictions,verbose = False))

RMSE on training data :  1.1973073332039819
RMSE on test data:  1.3675671978367903


In [ ]:
df_merged.loc[df_merged.customerid=='63d102996fee010ba791308c'].item_name.value_counts().sum()

18

In [ ]:
rating = Dataset.load_from_df(a, reader)

In [ ]:
item_ratings = train_test_split(rating, random_state = 42)

NameError: ignored

In [ ]:
svd_model = SVD(random_state = 42)
svd_model.fit(train_ratings)

In [ ]:
from surprise import SVD

In [ ]:
train_predictions = svd_model.test(train_ratings.build_testset())
test_predictions = svd_model.test(test_ratings)
print("RMSE on training data : ", accuracy.rmse(train_predictions, verbose = False))
print("RMSE on test data: ", accuracy.rmse(test_predictions, verbose = False))

RMSE on training data :  0.8738979704330262
RMSE on test data:  1.3076715202890457


In [ ]:
import pickle

# Load saved model from pickle file
with open('/content/drive/MyDrive/Trigital /svd_model.pkl', 'wb') as f:
    pickle.dump(svd_model, f)

In [ ]:
item_predictions = baseline_model.test(train_ratings.build_testset())

In [ ]:
def get_top_n_recommendations(customerid,predictions, n=11569):
    predict_ratings = {}
    # loop for getting predictions for the user
    for uid, iid, true_r, est, _ in predictions:
        if (uid==customerid):
            predict_ratings[iid] = est
    predict_ratings = sorted(predict_ratings.items(), key=lambda kv: kv[1],reverse=True)[:n]
    top_items = [i[0] for i in predict_ratings]
    # print(top_items)
    # top_items = [str(i) for i in top_items]
    # print("="*10,"Recommended movies for user {} :".format(customerid),"="*10)
    # print(df_merged[df_merged["id"].isin(top_items)]["item_name"].to_string(index=False))
    return data_merged[data_merged["id"].isin(top_items)][["item_name","market","price","avgrating",'state']].sort_values(by=['avgrating'],ascending=False)
get_top_n_recommendations('633434892288e00638e17613',item_predictions).head()

,item_name,market,price,avgrating,state
3038,AbhisarikaFabulousGreyWomenKurtaSets,Clothes & Fashion,1899.0,5,Telangana
847,Coffee,Food & Restaurants,80.0,5,Telangana
3135,LimeGlycerinSoap100Gms,Grocery,95.0,5,Telangana
980,TieWhiteSatinSkinnyTie,Clothes & Fashion,299.0,5,Telangana
3110,AdrikaGracefulBlackWomenKurtaSets,Clothes & Fashion,1799.0,5,Telangana


## New user recommendation

In [ ]:
# new_user_id = 'new_user'
new_user = [('new_user',7399, 4), ('new_user',754, 5), ('new_user',7400, 3)]

new_user_predictions = baseline_model.test(new_user)



In [ ]:
get_top_n_recommendations('new_user',new_user_predictions)

,item_name,market,price,avgrating,state
2,Jeera,Grocery,1.0,4,Telangana
3,Roti,Food & Restaurants,35.0,4,NaN
1,WingsBiryani,Food & Restaurants,339.0,1,NaN


## Trending items

In [ ]:
from collections import defaultdict

In [ ]:
# item_popularity = defaultdict(int)
# for user, item, rating in train_ratings.all_ratings():
#     item_popularity[item] += 1

# # Get the top-N trending items
# n = 10 # number of items to recommend
# top_n = sorted(item_popularity.items(), key=lambda x: -x[1])[:n]

# # Print the top-N trending items
# print('Top-N trending items:')
# for item_id, _ in top_n:
#     print(df_merged.loc[df_merged.id==item_id].item_name)

In [ ]:
data=get_top_n_recommendations('62d7e395ec76434350c3b45c',item_predictions)

In [ ]:
g1=data.groupby(['market'])

In [ ]:
def best_pick(item):
  df_item=g1.get_group(item)
  final_item=df_item[['avgrating','item_name','price',]]
  best_pick_item=final_item.sort_values(by='avgrating',ascending=False)
  best_pick_item.drop_duplicates('item_name',inplace=True)
  return best_pick_item.head()

In [ ]:
best_pick('Virtual assets & Real Estate')

KeyError: ignored

In [ ]:
from surprise.model_selection import GridSearchCV,RandomizedSearchCV

In [ ]:
param_distributions  = {'n_factors': list(range(50,160,10)),'reg_all': np.arange(0.02,0.1),'n_epochs' : list(range(1,51))}
rs = GridSearchCV(SVD, param_distributions, measures=['rmse'], return_train_measures = True, cv = 5)
rs.fit(ratings)
rs.best_params['rmse']

{'n_factors': 150, 'reg_all': 0.02, 'n_epochs': 50}

In [ ]:
tuned_svd_model = SVD(n_factors=130, reg_all =0.1, n_epochs = 50, random_state = 42,verbose = False)
tuned_svd_model.fit(train_ratings)
train_predictions = tuned_svd_model.test(train_ratings.build_testset())
test_predictions = tuned_svd_model.test(test_ratings)
print("RMSE on training data : ", accuracy.rmse(train_predictions, verbose = False))
print("RMSE on test data: ", accuracy.rmse(test_predictions, verbose = False))

RMSE on training data :  0.45180141635725146
RMSE on test data:  1.287761933127254


In [ ]:
final_df.iloc[0:1000].head()

,customerid,id,avgrating
0,628cb023c3ccbfbb1d5f410e,1,1
1,633ac3aff0f1d2a7c6a29f5a,7399,1
2,630718b7cf57702ea522b0cb,754,4
3,63419766f0f1d24d99a34f54,7400,4
4,62dbd5ce9670f2007f7cce70,809,3
